# What is the relationship between stress levels and physical activity in participants?

### Name:
    Samaneh Shahpouri
### Student number:
    460145

# Introduction:

The relationship between stress levels and physical activity has been widely studied, with a significant body of evidence suggesting that physical activity can help to prevent, as well as improve, general physical and mental well-being in the context of stress. However, the specific relationship between stress levels and physical activity in different geographical regions is not well understood. The objective of the current study is to examine the relationship between stress levels and physical activity in participants from different geographical regions within the Lifelines cohort.

This study will make use of data from the Lifelines cohort, a large-scale population-based study that has been collecting data on the health and well-being of individuals living in the northern Netherlands since 2006. By examining the relationship between stress levels and physical activity in participants from different geographical regions within the Lifelines cohort, this study aims to provide a more nuanced understanding of the relationship between these two factors and how it may vary across different populations. The results of this study will be of great importance for public health and well-being, as they will inform the development of effective interventions aimed at reducing the negative effects of stress and promoting physical activity.






# About the data



## **Stress**

Data on past-year stress exposure (section: mental health) at baseline and follow-up (3 time points: follow-up questionnaire 1 and 2 and questionnaire at second assessment).
Two stress exposure questionnaires were administered: the List of Threatening Experiences (LTE) measuring acute stressful life events (or SLE), and the Long-term Difficulties Inventory
(LDI) measuring long-term difficulties (LTD) i.e. more chronic stressors. The questionnaires were part of the questionnaire packs to be administered at home. Both concern stressors in the past year. The validation and the reliability of the LDI was performed in a study by Rosmalen et al.
The LTE lists 12 stressful life events (e.g. death of a relative, serious disease) and asks whether participants experienced such an event in the past year (no=2, yes=1). Items 13 (“did you
experience any other major life events in the past year?”) and 13A (“can you briefly describe this event?”) are discarded.
The LDI lists 12 potential sources of chronic stress (e.g. financial difficulties, work-related stress, strained relationships) and asks how much the participant was affected by this type of stress (not=1, somewhat=2, much=3). Variable Label

**LTE_SUM_T1** Total number of stressful life events that happened to participant in the past year at baseline as mean (T1)

**LDI_SUM_T1** Total amount of stress from long-term/chronic stressors experienced by the participant at baseline as mean (T1)

**LTE_SUM_T2** Total number of stressful life events that happened to participant in the past year at second assessment as mean (T2)

**LDI_SUM_T2** Total amount of stress from long-term/chronic stressors experienced by the participant at second assessment as mean (T2)

    Since we are not studying temporal changes in this study, we will not be using T2 data, which may be useful for further research.

## **Physical Activity**


Based on the raw data from the SQUASH instrument in 1A Questionnaire 2, the sum scores for
physical activity per activity type and overall were calculated for all adult participants who
sufficiently filled in the questionnaire (sections: lifestyle & environment and secondary & linked
variables).


**MWK_VAL** Minutes of weekly physical activity on moderate and vigorous intensity level
at baseline as mean (T1)

**SCOR_VAL** Score for weekly physical activity on moderate and vigorous intensity level,
based on the sum of minutes per activity times the intensity of the specific
activity at baseline as mean (T1)

**MWK_NO_VAL** Minutes of weekly physical activity on moderate and vigorous intensity level,
in leisure time and commuting domains (but not occupational) at baseline as
mean (T1

**SCOR_NO_VAL** Score for weekly physical activity on moderate and vigorous intensity level,
in leisure time and commuting domains (but not occupational), based on the
sum of minutes per activity times the intensity of the specific activity at
baseline as mean (T1)

**SPORTS_T1**  Minutes of weekly excercise as sport (T1).




Initially, our intention was to only investigate the correlation between physical activity and stress levels. However, after conducting further analysis of the data, we realized that body mass index (BMI) was also a significant factor affecting physical activity. Therefore, we have decided to include BMI as a personal characteristic in our study for further investigation.

# Import libreries

In [ ]:
# Importing necessary libraries for data processing and visualization
import pandas as pd # collection of functions for data processing and analysis modeled after R dataframes with SQL like features
import numpy as np  # foundational package for scientific computing
import re           # Regular expression operations
import matplotlib.pyplot as plt # Collection of functions for scientific and publication-ready visualization


# Importing Plotly libraries for interactive and dynamic visualizations
import plotly.offline as py     # Open source library for composing, editing, and sharing interactive data visualization 
from matplotlib import pyplot as pp
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# Importing Seaborn library for statistical visualizations
import seaborn as sns  # Visualization library based on matplotlib, provides interface for drawing attractive statistical graphics

# Importing yaml library for processing yaml files
import yaml

# Suppressing warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import pandas as pd # collection of functions for data processing
import numpy as np  # foundational package for scientific computing
import matplotlib.pyplot as plt # Collection of functions for visualization
import seaborn as sns  # Visualization library based on matplotlib
import yaml # Importing yaml library for processing yaml files

    
import numpy as np
from scipy.stats import iqr # iqr is the Interquartile Range function
import matplotlib.pyplot as plt


from scipy.stats import norm
import matplotlib.pyplot as plt
import numpy as np

# Load the Data

In [ ]:
# Load the configuration file
with open("config.yml", "r") as reader:
    config = yaml.safe_load(reader)

# Load data from the configuration
file = config['path']
df_zipcode = pd.read_csv(file, delimiter='\t', encoding='utf-16le')

file = config['path_general']
df_zipcode_general= pd.read_csv(file, delimiter='\t', encoding='utf-16le')

# Clean the Data

In [ ]:
# Select appropriate columns from the dataframe.
new_general = df_zipcode_general[['ZIPCODE', 'MWK_VAL', 'SCOR_VAL',
                                  'MWK_NO_VAL', 'SCOR_NO_VAL', 'SPORTS_T1',
                                  'LTE_SUM_T1', 'LDI_SUM_T1']]

# Select appropriate columns from the dataframe.
new_zip = df_zipcode[['ZIPCODE', 'GENDER', 'AGE_T1',
             'MWK_VAL', 'SCOR_VAL', 'MWK_NO_VAL', 'SCOR_NO_VAL', 
             'SPORTS_T1','LTE_SUM_T1', 'LDI_SUM_T1', 'BMI_T1']]

# Change to desire format.
new_zip.iloc[:, 2:] = new_zip.iloc[:, 2:].apply(lambda x: x.str.replace(',', '.')).astype(float)

# Merge for obtaining zipcodes.
new_df = new_general.merge(new_zip,on= 'ZIPCODE', how = 'inner')

# Remove duplicate columns.
new_df = new_df.drop(new_df.columns[new_df.columns.str.contains('_x')], axis=1)

# Remove "_y" from header names
new_df.rename(columns={col: col.rstrip('_y') for col in new_df.columns}, inplace=True)


gender = new_df['GENDER']
cond_list = [gender==1, gender==2]
choice_list = ["Male", "Female"]

new_df['GENDER'] = np.select(cond_list, choice_list)
new_df['GENDER'] = new_df['GENDER'].astype("category")
df = new_df


df.head()

In [ ]:
# Information about data set
df.info()

There is no missing value. Time for further investigation about data.


-------------------------------------


# Descriptive Summary


## Histogram


In [ ]:
columns = df.columns.tolist()
exclude = ['ZIPCODE', 'GENDER', 'AGE_T1']
num_plots = len([c for c in columns if c not in exclude])

fig, axs = plt.subplots(2, 4, figsize=(20, 10))
axs = axs.ravel()

for i, column in enumerate([c for c in columns if c not in exclude]):
    y = df[column]
    axs[i].hist(y, density=True, color='mistyrose')
    mean = y.mean()
    axs[i].axvline(mean, color='r', linestyle='dashed', linewidth=1)
    robust = y.median()
    axs[i].axvline(robust, color='b', linestyle='dashed', linewidth=1)
    x = np.linspace(y.min()-1, y.max(), 1000)
    mu, std = norm.fit(y)
    yr = norm.pdf(x, mu, std)
    axs[i].plot(x, yr, 'silver', linewidth=2)
    axs[i].set_title(column)

plt.tight_layout()
plt.show()

In [ ]:
def descriptive(df):
    desc = df.drop(['GENDER', 'ZIPCODE', 'AGE_T1'],
     axis=1).describe().round(1).drop(['count', 'std', '50%'], axis=0)
    i = -0.1
    j = 0
    Row = int(round(len(desc.columns.tolist())/2 + 0.1))
    f, ax = plt.subplots(Row, 2, figsize=(30, 15))
    colors = ['turquoise', 'tomato', 'limegreen','purple' , 'gold', 'sandybrown', 'orchid', 'dodgerblue']
    c = 0
    for name in desc.columns.tolist():
        desc[name].plot(kind='barh', figsize=(8, 7), title=name, ax=ax[round(i), j], fontsize=8, color=colors[c%len(colors)])
        for k, v in enumerate(desc[name].tolist()):
            ax[round(i), j].text(v, k-0.1, str(v), color='black', size=8)
            ax[round(i), j].title.set_size(8) # set font size for title

        i += 0.5
        if j == 0:
            j = 1
        else:
            j = 0
        c += 1
    f.tight_layout()

descriptive(df)


## Q-Q plot
A Q-Q (quantile-quantile) plot is a graphical tool used to compare two probability distributions by plotting their quantiles against each other. By creating this Q-Q plot, we can quickly and easily assess whether the data is approximately normally distributed and compare the distribution of samples to the normal distribution. It also allows us to visualize the 95% confidence interval of the sample data.

If the data points align closely with the normal line, it suggests that the data is approximately normally distributed. If the data points deviate significantly from the line, it suggests that the data is not normally distributed.



In [ ]:
def DS_Q_Q_Plot(y, est = 'robust', column = '',ax=None, **kwargs):
    """
      Estimated mu, sigma, n, and expected number of datapoints
      outside CI in Q-Q-plot.
    Author:            M.E.F. Apol
    """

    mu_0 = kwargs.get('mu', None)
    sigma_0 = kwargs.get('sigma', None)
    
    n = len(y); y_os = np.sort(y)
    mu_ML = np.mean(y); sigma2_ML = np.var(y); sigma_ML = np.std(y) 
    s2 = np.var(y, ddof=1); s = np.std(y, ddof=1); mu_R = np.median(y)
    sigma_R = iqr(y)/1.349

    # Assign values of mu and sigma for z-transform:
    if est == 'ML':
        mu, sigma = mu_ML, s
    elif est == 'robust':
        mu, sigma = mu_R, sigma_R
    elif est == 'preset':
        mu, sigma = mu_0, sigma_0
    else:
        print('Wrong estimation method chosen!')
        return()
        
    n_dev = np.round(0.05*n); z_i = (y_os - mu)/sigma
    i = np.array(range(n)) + 1; p_i = (i - 0.5)/n

    z_th = norm.ppf(p_i, 0, 1)
    SE_z_th = (1/norm.pdf(z_th, 0, 1)) * np.sqrt((p_i * (1 - p_i)) / n)
    CI_upper = z_th + 1.96 * SE_z_th; CI_lower = z_th - 1.96 * SE_z_th


    plt.tight_layout()
    plt.plot(z_th, z_i, 'o', color='k', label='experimental data')
    plt.plot(z_th, z_th, '--', color='r', label='normal line')
    plt.plot(z_th, CI_upper, '--', color='b', label='95% CI')
    plt.plot(z_th, CI_lower, '--', color='b')
    plt.xlabel('Theoretical quantiles, $z_{(i)}$')
    plt.ylabel('Sample quantiles, $z_i$')
    plt.title('Q-Q plot (' + est + ') Data: ' + column, size = 10 )
    plt.legend(loc='best'); plt.show();
    print('Estimation method: ' + est)
    print('n = {:d}, mu = {:.4g}, sigma = {:.4g}'.format(n, mu,sigma))
    
    # Expected number of deviations (95% confidence level):
    n_dev = np.round(0.05*n)
    
    print('Expected number of data outside CI: {:.0f}'.format(n_dev))
    pass


In [ ]:
import ipywidgets as widgets

def choose_column(column):
    y = df[column]
    DS_Q_Q_Plot(y, est='robust', column=column)
    plt.tight_layout()
    plt.show()

columns = df.columns.tolist()
exclude = ['ZIPCODE', 'GENDER']
y_columns = [c for c in columns if c not in exclude]

widgets.interact(choose_column, column=widgets.Dropdown(
    options=y_columns, description='Dataset:'));


Almost 33 data points out of more than 650 data fall outside of the confidence interval, but this is not necessarily important and we can assume that the data is normally distributed.

--------------------------------------

## Visualize the Data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume the data is stored in a pandas dataframe named "df"

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot SCOR_VAL and SCOR_NO_VAL
male = df[df['GENDER'] == 'Male']
female = df[df['GENDER'] == 'Female']
ax1.scatter(male['SCOR_VAL'], male['LDI_SUM_T1'], c='blue', label='Male')
ax1.scatter(female['SCOR_VAL'], female['LDI_SUM_T1'], c='red', label='Female')
ax1.set_xlabel('SCOR_VAL')
ax1.set_ylabel('LDI_SUM_T1')
ax1.legend()

# Plot LTE_SUM_T1 and LTE_SUM_T2
ax2.scatter(male['SCOR_VAL'], male['LTE_SUM_T1'], c='blue', label='Male')
ax2.scatter(female['SCOR_VAL'], female['LTE_SUM_T1'], c='red', label='Female')
ax2.set_xlabel('SCOR_VAL')
ax2.set_ylabel('LTE_SUM_T1')
ax2.legend()

plt.show()


In [ ]:
# Assume the data is stored in a pandas dataframe named "df"

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot SCOR_VAL and SCOR_NO_VAL
male = df[df['GENDER'] == 1]
female = df[df['GENDER'] == 2]
ax1.scatter(male['MWK_VAL'], male['LTE_SUM_T1'], c='blue', label='Male')
ax1.scatter(female['MWK_VAL'], female['LTE_SUM_T1'], c='red', label='Female')
ax1.set_xlabel('MWK_VAL')
ax1.set_ylabel('LTE_SUM_T1')
ax1.legend()

# Plot LTE_SUM_T1 and LTE_SUM_T2
ax2.scatter(male['MWK_NO_VAL'], male['LDI_SUM_T1'], c='blue', label='Male')
ax2.scatter(female['MWK_NO_VAL'], female['LDI_SUM_T1'], c='red', label='Female')
ax2.set_xlabel('MWK_NO_VAL')
ax2.set_ylabel('LDI_SUM_T1')
ax2.legend()

plt.show()


## Categorize the Data

### Body Mass Index
BMI is a person’s weight in kilograms divided by the square of height in meters. A high BMI can indicate high body fatness.

To calculate BMI, see the Adult BMI Calculator or determine BMI by finding your height and weight in this BMI Index Chart.

If your BMI is less than 18.5, it falls within the **underweight** range.

If your BMI is 18.5 to <25, it falls within the **healthy** weight range.

If your BMI is 25.0 to <30, it falls within the **overweight** range.

If your BMI is 30.0 or higher, it falls within the **obesity** range.


So, we can make categorize the BMI data.


In [ ]:
bmi = df['BMI_T1']
cond_list = [bmi < 18.5, bmi < 25, bmi < 30, bmi >= 30]
choice_list = ["Underweight", "Healthy", "Overweight", "Obese"]

df['BMI_LEV'] = np.select(cond_list, choice_list)
df['BMI_LEV'] = df['BMI_LEV'].astype("category")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume the data is stored in a pandas dataframe named "df"

# Map the "BMI level" column to circle size values



# Assume the data is stored in a pandas dataframe named "df"

size_mapping = {"Underweight": 10, "Healthy": 20, "Overweight": 100, "Obese": 150}
df['size'] = df['BMI_LEV'].map(size_mapping)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Create custom markers for each size
markers = []
sizes = list(size_mapping.values())
for size in sizes:
    marker = plt.scatter([],[], s=size, c='gray', alpha=0.5)
    markers.append(marker)

# Plot SCOR_VAL and SCOR_NO_VAL
male = df[df['GENDER'] == 'Male']
female = df[df['GENDER'] == 'Female']
ax1.scatter(male['SCOR_VAL'], male['LDI_SUM_T1'], label='Male', s=male['size'], alpha=0.4, color='dodgerblue')
ax1.scatter(female['SCOR_VAL'], female['LDI_SUM_T1'], label='Female', s=female['size'],  alpha=0.4, color='hotpink')
ax1.set_xlabel('SCOR_VAL')
ax1.set_ylabel('LDI_SUM_T1')

# Add the size information to the legend
labels = list(size_mapping.keys())
legend1 = ax1.legend(markers, labels, loc='best', title='BMI level')
ax1.add_artist(legend1)

# Add the gender information to the legend
legend2 = ax1.legend(loc='upper left', title='Gender')

# Plot LTE_SUM_T1 and LTE_SUM_T2
ax2.scatter(male['SCOR_VAL'], male['LTE_SUM_T1'], label='Male', s=male['size'], alpha=0.4, color='dodgerblue')
ax2.scatter(female['SCOR_VAL'], female['LTE_SUM_T1'], label='Female', s=female['size'],  alpha=0.4, color='hotpink')
ax2.set_xlabel('SCOR_VAL')
ax2.set_ylabel('LTE_SUM_T1')

# Add the size information to the legend
legend1 = ax2.legend(markers, labels, loc='best', title='BMI level')
# ax2.add_artist(legend1)

# Add the gender information to the legend
legend2 = ax2.legend(loc='upper left', title='Gender')

plt.show()




## Stress Index
The List of Threatening Experiences (LTE) is a reliable and valid measure of stress in mental health, and the strength of its association with mental disorders depends on the method used to quantify the LTE scores.

The validated List of Threatening Events (LTE) and Long-term Difficulties Inventory (LDI) were utilized in the past to measure stress, with higher scores indicating higher levels of stress. The LTE consists of 12 major categories of stressful life events, with a range sum score of 0 to 12, while the LDI measures exposure to long-term difficulties in 12 life domains, with a range sum score of 0 to 24.

The total scores were divided into various categories, for example:

LTE: 0, 1, 2, and ≥3
LDI: 0, 1-2, 3-4, and ≥5

In the study, items were scored on a two-point Likert scale.

    Short-term stress (LTE): Mild (≤1.5), Severe (>1.5)
    Long-term stress (LDI): Mild (≤2.5), Severe (>2.5)

In [ ]:
df['LTE_LEV'] = ''
lte = df['LTE_SUM_T1']
cond_list = [lte <= 1.50, lte > 1.50]
choice_list = ["Mild", "Severe"]
df['LTE_LEV'] = np.select(cond_list, choice_list)
df['LTE_LEV'] = df['LTE_LEV'].astype("category")

df['LDI_LEV'] = ''
ldi = df['LDI_SUM_T1']
cond_list = [ldi <= 2.50, ldi > 2.50]
choice_list = ["Mild", "Severe"]
df['LDI_LEV'] = np.select(cond_list, choice_list)
df['LDI_LEV'] = df['LDI_LEV'].astype("category")
df.head()

In [ ]:
# Separate the data into male and female participants
male = df[df['GENDER'] == 1]
female = df[df['GENDER'] == 2]

# Plot the stress level (LDI_SUM_T1) against age (AGE_CAT) for male and female participants
plt.scatter(male['AGE_T1'], male['MWK_VAL'], label='Male', alpha=0.4, color='dodgerblue')
plt.scatter(female['AGE_T1'], female['MWK_VAL'], label='Female', alpha=0.4, color='hotpink')
# Set the x and y axis limits based on the minimum and maximum values of the data
plt.xlim(df['AGE_T1'].min() - 10, df['AGE_T1'].max() + 10)
plt.ylim(df['MWK_VAL'].min() - 100, df['MWK_VAL'].max() + 100)

# Add labels and a legend to the plot
plt.xlabel('Age')
plt.ylabel('Physical Activity (MWK_VAL)')
plt.legend()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume the data is stored in a pandas dataframe named "df"

# Map the "BMI level" column to circle size values



# Assume the data is stored in a pandas dataframe named "df"

size_mapping = {"Underweight": 10, "Healthy": 20, "Overweight": 100, "Obese": 150}
df['size'] = df['BMI_LEV'].map(size_mapping)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Create custom markers for each size
markers = []
sizes = list(size_mapping.values())
for size in sizes:
    marker = plt.scatter([],[], s=size, c='gray', alpha=0.5)
    markers.append(marker)

# Plot SCOR_VAL and SCOR_NO_VAL
male = df[df['GENDER'] == 'Male']
female = df[df['GENDER'] == 'Female']
ax1.scatter(male['SCOR_VAL'], male['LDI_SUM_T1'], label='Male', s=male['size'], alpha=0.4, color='dodgerblue')
ax1.scatter(female['SCOR_VAL'], female['LDI_SUM_T1'], label='Female', s=female['size'],  alpha=0.4, color='hotpink')
ax1.set_xlabel('SCOR_VAL')
ax1.set_ylabel('LDI_SUM_T1')

# Add the size information to the legend
labels = list(size_mapping.keys())
legend1 = ax1.legend(markers, labels, loc='best', title='BMI level')
ax1.add_artist(legend1)

# Add the gender information to the legend
legend2 = ax1.legend(loc='upper left', title='Gender')

# Plot LTE_SUM_T1 and LTE_SUM_T2
ax2.scatter(male['SCOR_VAL'], male['LTE_SUM_T1'], label='Male', s=male['size'], alpha=0.4, color='dodgerblue')
ax2.scatter(female['SCOR_VAL'], female['LTE_SUM_T1'], label='Female', s=female['size'],  alpha=0.4, color='hotpink')
ax2.set_xlabel('SCOR_VAL')
ax2.set_ylabel('LTE_SUM_T1')

# Add the size information to the legend
legend1 = ax2.legend(markers, labels, loc='best', title='BMI level')
# ax2.add_artist(legend1)

# Add the gender information to the legend
legend2 = ax2.legend(loc='upper left', title='Gender')

plt.show()


In [ ]:
# Separate the data into male and female participants
male = df[df['GENDER'] == 1]
female = df[df['GENDER'] == 2]

# Plot the stress level (LDI_SUM_T1) against age (AGE_CAT) for male and female participants
plt.scatter(male['AGE_T1'], male['LDI_SUM_T1'], label='Male', alpha=0.4, color='dodgerblue')
plt.scatter(female['AGE_T1'], female['LDI_SUM_T1'], label='Female', alpha=0.4, color='hotpink')
# Set the x and y axis limits based on the minimum and maximum values of the data
plt.xlim(df['AGE_T1'].min() - 1, df['AGE_T1'].max() + 1)
plt.ylim(df['LDI_SUM_T1'].min() - 1, df['LDI_SUM_T1'].max() + 1)

# Add labels and a legend to the plot
plt.xlabel('Age')
plt.ylabel('Stress Level (LDI_SUM_T1)')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# The easiest way for plotting boxplot.
import seaborn as sns
male = df[df['GENDER'] == 1]
female = df[df['GENDER'] == 2]

df['Stress level'] = 'Normal'
df.loc[df['LDI_SUM_T1'] > 2, 'Stress level'] = 'Upper than normal'


#sns.boxplot(x='Stress level', y='MWK_NO_VAL', hue = 'GENDER', data=df)
sns.violinplot(x='Stress level', y='MWK_NO_VAL', hue = 'GENDER', palette=sns.color_palette("pastel"), data = df, split = True)
#order=['20 or less', '21 to 35', '36 to 50', '51 or more']

In [ ]:
x = 'LDI_SUM_T1'
data_range = df[x].max() - df[x].min()
print (df[x].max(),  df[x].min())
print(data_range)
df[x].median()


## hypothesis

## Box Plot

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))


sns.boxplot(x='LDI_LEV', y='MWK_NO_VAL',palette='husl', hue = 'GENDER', data=df, ax=ax1)
ax1.set_xlabel('long term Stress')
ax1.set_ylabel('Minutes of weekly physical activity in leisure time')


sns.boxplot(x='LDI_LEV', y='MWK_VAL', data=df, palette='husl', hue = 'GENDER', ax=ax2)
ax2.set_xlabel('Short term Stress')
ax2.set_ylabel('Minutes of weekly physical activity on high intensity level')

sns.boxplot(x='LDI_LEV', y='SPORTS_T1', data=df, palette='husl', hue = 'GENDER', ax=ax3)
ax3.set_xlabel('Short term Stress')
ax3.set_ylabel('Minutes of weekly Sport')


plt.tight_layout()
plt.show()

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

df['Stress level'] = 'Normal'
df.loc[df['LDI_SUM_T1'] > 2.5, 'Stress level'] = 'Upper than normal'

sns.boxplot(x='Stress level', y='SPORTS_T1', data=df, ax=ax1)
ax1.set_xlabel('Stress level')
ax1.set_ylabel('SPORTS_T1')

sns.boxplot(x='Stress level', y='SCOR_VAL', data=df, palette='husl', ax=ax2)
ax2.set_xlabel('Stress level')
ax2.set_ylabel('SCOR_VAL')

plt.tight_layout()
plt.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def plot_selector(plot_type):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    
    

    if plot_type == 'LTE_SUM_T1':
        df['Stress level'] = 'Normal'
        df.loc[df['LTE_SUM_T1'] > 1.5, 'Stress level'] = 'Upper than normal'
        sns.boxplot(x='Stress level', y='SCOR_VAL', data=df,hue = 'GENDER',  palette='Set1',color='gray', ax=ax1)
        sns.boxplot(x='Stress level', y='SCOR_NO_VAL', data=df,hue = 'GENDER',  palette='Set2',color='gray', ax=ax2)
        ax1.set_xlabel('Stress level')
        ax1.set_ylabel('SCOR_NO_VAL')
        ax2.set_ylabel('SCOR_VAL')
    elif plot_type == 'LDI_SUM_T1':
        df['Stress level'] = 'Normal'
        df.loc[df['LDI_SUM_T1'] > 2.5, 'Stress level'] = 'Upper than normal'
        sns.boxplot(x='Stress level', y='SCOR_VAL', data=df, hue = 'GENDER', palette='Set1',color='gray', ax=ax1)
        sns.boxplot(x='Stress level', y='SCOR_NO_VAL', data=df, hue = 'GENDER', palette='Set2',color='gray', ax=ax2)
        ax2.set_xlabel('Stress level')
        ax1.set_ylabel('SCOR_NO_VAL')
        ax2.set_ylabel('SCOR_VAL')
    else: 
        pass

    plt.tight_layout()
    plt.show()

plot_select = widgets.Dropdown(
    options=['LTE_SUM_T1', 'LDI_SUM_T1'],
    value='LTE_SUM_T1',
    description='Plot Type:',
    disabled=False,
)

widgets.interact(plot_selector, plot_type=plot_select)
display()




## Heatmap Visualization

The pairwise Pearson's correlation coefficients between all variables. The correlation coefficients are then visualized as a heatmap.
The purpose of this code is to visualize the relationships between variables in the dataframe in a quick and easy-to-understand way.

In this heatmap, the closer the correlation coefficient is to 1, the stronger the positive linear relationship between two variables, and the closer the correlation coefficient is to -1, the stronger the negative linear relationship between two variables. A correlation coefficient of 0 indicates no linear relationship between two variables.
In general, a correlation coefficient of 0.1 is considered a small correlation, a coefficient of 0.3 is considered a moderate correlation, and a coefficient of 0.5 or higher is considered a strong correlation. However, these cut-offs are not absolute and can vary based on the context of the data and the field of study.
A negative sign indicates a negative linear relationship, meaning that as one variable increases, the other variable decreases.

In [ ]:
sns.heatmap(df.corr(), cmap="YlGnBu", annot=True)
plt.show()

**Results**

The heatmap provides a visual representation of the correlation matrix of the variables in the dataset. From the heatmap, we can observe that the variable "LTE" (short-term stress) has a moderate positive correlation with "LDI" (long-term stress, r = 0.47) and a moderate positive correlation with "BMI" (body mass index, r = 0.23). These correlations suggest that individuals who experience more short-term stress tend to have higher levels of long-term stress and a higher body mass index.

In addition, the heatmap also shows that the variable "LDI" (long-term stress) has a moderate negative correlation with "Age" (r = -0.47), a moderate negative correlation with "MWK" (minutes of physical activity in leisure time per week, r = -0.32), a moderate positive correlation with "LTE" (short-term stress, r = 0.47), a moderate negative correlation with "BMI" (r = -0.3), and a small positive correlation with "Sport" (minutes of sport per week, r = 0.24). These correlations suggest that individuals who experience more long-term stress tend to be younger, engage in less physical activity during leisure time, and have a lower body mass index, while also engaging in a slightly higher amount of sport activity.

It is important to note that the correlations presented here are only indicative of linear relationships and do not necessarily imply causality. Further analysis,  may be necessary to determine the effect of the variables on each other.

## Statistical Tests

For determining the impact of stress level (severe versus mild) on physical activity in individuals over a long-term period, we use the two-sample t-test for means, also known as the independent samples t-test. This test is used to compare the means of two independent groups and is commonly utilized when there is a need to determine if a significant difference exists between the means of the two groups.

In [ ]:
def DS_2sample_ttest_means(y1, y2, equal_var=False, alternative='two-sided', alpha=0.05):
    """
    *
    Function DS_2sample_ttest_means(y1, y2, equal_var=False, alternative='two-sided', alpha=0.05)
    
       This function performs a 2-sample (Welch's) t-test (Null Hypothesis Significance Test) 
       in the spirit of R, testing 2 averages with *unknown* standard deviation.
       The function also evaluates the effect size (Cohen's d).
       
    Requires:          -
       
    Usage:             DS_2sample_ttest_means(y1, y2, 
                            alternative=['two-sided']/'less'/'greater',
                            equal_var=[False]/True, alpha = 0.05)
     
                         alternative = 'two-sided' [default]  H1: mu_1 != mu_2
                                       'less'                 H1: mu_1 < mu_2
                                       'greater'              H1: mu_1 > mu_2
                         equal_var = False                    perform Welch t-test
                                     True                     perform 2-sample t-test
                         alpha:   significance level of test [default: 0.05]
     
    Return:            t, p-value, t.crit.L, t.crit.R  [ + print interpretable output to stdout ]
                       where t.crit.L and t.crit.R are the lower and upper critical values, 
                       t is the test statistic and p-value is the p-value of the test.     
     
    Author:            M.E.F. Apol
    Date:              2022-01-28, rev. 2022_08_26
    Validation:
    """
    
    from scipy.stats import ttest_ind
    from scipy.stats import t as t_distr
    import numpy as np
    
    t, p_samp = ttest_ind(y1, y2, equal_var = equal_var)
    y_av_1 = np.mean(y1)
    y_av_2 = np.mean(y2)
    n_1 = len(y1)
    n_2 = len(y2)
    s2_1 = np.var(y1, ddof=1)
    s2_2 = np.var(y2, ddof=1)
    print(80*'-')
    if equal_var == True:
        print('2-sample t-test for 2 means:')
        print('     assuming Normal(mu.1, sigma2) data for dataset 1')
        print('     assuming Normal(mu.2, sigma2) data for dataset 2')
        df = n_1 + n_2 - 2
    else:
        print('Welch t-test for 2 means:')
        df = (s2_1/n_1 + s2_2/n_2)**2 / ( 1/(n_1-1)*(s2_1/n_1)**2 + 1/(n_2-1)*(s2_2/n_2)**2 )
        print('     assuming Normal(mu.1, sigma2.1) data for dataset 1')
        print('     assuming Normal(mu.2, sigma2.2) data for dataset 2')
    print('y.av.1 = {:.3g}, y.av.2 = {:.3g}, s2.1 = {:.3g}, s2.2 = {:.3g}, n.1 = {:d}, n.2 = {:d}, alpha = {:.3g}'.format(y_av_1, y_av_2, s2_1, s2_2, n_1, n_2, alpha))
    print('H0: mu.1  = mu.2')
    if alternative == 'two-sided':
        print('H1: mu.1 != mu.2')
        p_value = p_samp
        t_crit_L = t_distr.ppf(alpha/2, df)
        t_crit_R = t_distr.ppf(1-alpha/2, df)      
    elif alternative == 'less':
        print('H1: mu.1  < mu.2')
        if t <= 0:
            p_value = p_samp/2
        else:
            p_value = 1 - p_samp/2
        t_crit_L = t_distr.ppf(alpha, df)
        t_crit_R = float('inf')
    elif alternative == 'greater':
        print('H1: mu.1  > mu.2')
        if t >= 0:
            p_value = p_samp/2
        else:
            p_value = 1 - p_samp/2
        t_crit_L = float('-inf')
        t_crit_R = t_distr.ppf(1-alpha, df)
    else:
        print('Wrong alternative hypothesis chosen!')
        print(80*'-' + '\n')
        t, p_value, t_crit_L, t_crit_R = np.nan, np.nan, np.nan, np.nan
        return(t, p_value, t_crit_L, t_crit_R)
    
    # Effect size (Cohen's d.av):
    d_av = t * np.sqrt(1/n_1 + 1/n_2)
    print('t = {:.4g}, p-value = {:.4g}, t.crit.L = {:.4g}, t.crit.R = {:.4g}, df = {:.4g}'.format(t, p_value, t_crit_L, t_crit_R, df))
    print('Effect size: d.av = {:.3g}; benchmarks |d.av|: 0.2 = small, 0.5 = medium, 0.8 = large'.format(d_av))
    print(80*'-' + '\n')
    return(t, p_value, t_crit_L, t_crit_R)

In [ ]:
df_mild_ldi = df[df["LDI_LEV"] == 'Mild']
df_severe_ldi = df[df["LDI_LEV"] == 'Severe']
y2 = df_mild_ldi["MWK_VAL"]
y1 = df_severe_ldi["MWK_VAL"]
DS_2sample_ttest_means(y1, y2, alternative='less')

**interpretation**

The result of the Welch t-test suggests that there is a significant difference in the mean minutes of physical activity between individuals who have experienced severe stress and those who have faced mild stress in the long-term.

The negative test statistic, t = -9.479, indicates that individuals with severe stress have a lower mean physical activity time compared to those with mild stress. This finding is supported by the p-value of 2.737e-20, which is less than the alpha level of 0.05, suggesting that the null hypothesis (H0: mu.1 = mu.2) can be rejected.

In essence, the results of the test show that individuals with severe stress tend to have a lower mean physical activity time compared to those with mild stress in the long-term, and this difference is considered substantial.

In [ ]:
df_mild_lte = df[df["LTE_LEV"] == 'Mild']
df_severe_lte = df[df["LTE_LEV"] == 'Severe']
y2 = df_mild_lte["MWK_VAL"]
y1 = df_severe_lte["MWK_VAL"]
DS_2sample_ttest_means(y1, y2, alternative='two-sided')

**interpretation**

The p-value is 0.7744, indicating the likelihood of observing a t-statistic value as extreme as -0.2959, given that the null hypothesis is true. When the p-value is greater than the significance level (α = 0.05), it means that we fail to reject the null hypothesis, indicating that there is not enough evidence to suggest a difference in the means of physical activity time in individuals with short-term stress levels of either mild or severe.

In [ ]:
import geopandas as gpd

# Load the shapefile into a GeoDataFrame
gdf = gpd.read_file("C:\zshahpouri\data\georef-netherlands-postcode-pc4.shp")
gdf = gdf.rename(columns = {'pc4_code':'ZIPCODE'})


gdf['ZIPCODE'] = gdf['ZIPCODE'].astype(float)
# Merge the data with the GeoDataFrame based on the city name
merged_gdf = gdf.merge(df, left_on='ZIPCODE', right_on='ZIPCODE')

# Plot the geo plot using the 'plot' method from GeoPandas
merged_gdf.plot(column='SCOR_VAL', cmap='RdYlGn', legend=True)




In [ ]:
import folium
import geopandas as gpd

# Load the shapefile into a GeoDataFrame
gdf = gpd.read_file("C:\zshahpouri\data\georef-netherlands-postcode-pc4.shp")
gdf = gdf.rename(columns = {'pc4_code':'ZIPCODE'})

# Merge the data with the GeoDataFrame based on the city name
merged_gdf = gdf.merge(df, left_on='ZIPCODE', right_on='ZIPCODE')

# Create a map object centered on the Netherlands
m = folium.Map(location=[52.3, 5.0], zoom_start=8)

# Create a GeoJSON object from the merged GeoDataFrame
geo_json = folium.GeoJson(
    merged_gdf.to_json(),
    style_function=lambda feature: {
        'fillColor': '#00ff00',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }
)

# Add the GeoJSON object to the map
geo_json.add_to(m)

# Loop through the data points and add markers for each one
for i, row in merged_gdf.iterrows():
    folium.CircleMarker(location=[row['geometry'].y, row['geometry'].x],radius=5,color='red',fill=True,fill_color='red',opacity=0.7,popup=str(row['SCOR_VAL'])).add_to(m)

            #Save the map
m.save("map.html")